In [1]:
import numba
import numpy as np
from scipy.integrate import odeint

In [2]:
# Motor
Rm = 8.4  # Resistance
kt = 0.042  # Current-torque (N-m/A)
km = 0.042  # Back-emf constant (V-s/rad)

# Rotary Arm
mr = 0.095  # Mass (kg)
Lr = 0.085  # Total length (m)
Jr = mr * Lr ** 2 / 12  # Moment of inertia about pivot (kg-m^2)

# Pendulum Link
mp = 0.024  # Mass (kg)
Lp = 0.129  # Total length (m)
Jp = mp * Lp ** 2 / 12  # Moment of inertia about pivot (kg-m^2)

Br = Dr = 0.0005  # Equivalent viscous damping coefficient (N-m-s/rad)
Bp = Dp = 0.00005  # Equivalent viscous damping coefficient (N-m-s/rad)
g = 9.81  # Gravity constant


In [3]:
def diff_forward_model_ode(state, t, action, dt):
    theta = state[0]
    alpha = state[1]
    theta_dot = state[2]
    alpha_dot = state[3]
    Vm = action
    tau = (km * (Vm - km * theta_dot)) / Rm  # torque

    # fmt: off
    alpha_dot_dot = (2.0*Lp*Lr*mp*(4.0*Dr*theta_dot + Lp**2*alpha_dot*mp*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*alpha_dot**2*mp*np.sin(alpha) - 4.0*tau)*np.cos(alpha) - 0.5*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp)*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha)))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
    theta_dot_dot = (-Lp*Lr*mp*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha))*np.cos(alpha) + (4.0*Jp + Lp**2*mp)*(4.0*Dr*theta_dot + Lp**2*alpha_dot*mp*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*alpha_dot**2*mp*np.sin(alpha) - 4.0*tau))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
    # fmt: on

    diff_state = np.array([theta_dot, alpha_dot, theta_dot_dot, alpha_dot_dot])
    diff_state = np.array(diff_state, dtype="float64")
    return diff_state


def forward_model_ode(theta, alpha, theta_dot, alpha_dot, Vm, dt, integration_steps):
    t = np.linspace(0.0, dt, integration_steps)

    state = np.array([theta, alpha, theta_dot, alpha_dot])
    next_state = np.array(odeint(diff_forward_model_ode, state, t, args=(Vm, dt))).reshape(4,)
    
    theta = next_state[0]
    alpha = next_state[1]
    theta_dot = next_state[2]
    alpha_dot = next_state[3]

    theta = ((theta + np.pi) % (2 * np.pi)) - np.pi
    alpha = ((alpha + np.pi) % (2 * np.pi)) - np.pi

    return theta, alpha, theta_dot, alpha_dot


In [4]:
def forward_model_euler(theta, alpha, theta_dot, alpha_dot, Vm, dt, integration_steps):
    dt /= integration_steps
    for step in range(integration_steps):
        tau = (km * (Vm - km * theta_dot)) / Rm  # torque

        # fmt: off
        alpha_dot_dot = (2.0*Lp*Lr*mp*(4.0*Dr*theta_dot + Lp**2*alpha_dot*mp*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*alpha_dot**2*mp*np.sin(alpha) - 4.0*tau)*np.cos(alpha) - 0.5*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp)*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha)))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
        theta_dot_dot = (-Lp*Lr*mp*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha))*np.cos(alpha) + (4.0*Jp + Lp**2*mp)*(4.0*Dr*theta_dot + Lp**2*alpha_dot*mp*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*alpha_dot**2*mp*np.sin(alpha) - 4.0*tau))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
        # fmt: on

        theta_dot += theta_dot_dot * dt
        alpha_dot += alpha_dot_dot * dt

        theta += theta_dot * dt
        alpha += alpha_dot * dt

        theta %= 2 * np.pi
        alpha %= 2 * np.pi

    return theta, alpha, theta_dot, alpha_dot


In [5]:
def f(forward_model, n=1000):
    for i in range(n):
        theta, alpha, theta_dot, alpha_dot = np.random.randn(4,) * 0.1
        Vm = np.random.randn()
        theta, alpha, theta_dot, alpha_dot = forward_model(
            theta, alpha, theta_dot, alpha_dot, Vm, dt=0.001, integration_steps=1
        )


# Without numba

In [6]:
%timeit -n 100 f(forward_model_ode)

41.4 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit -n 100 f(forward_model_euler)

54.9 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# With numba

In [8]:
forward_model_ode = numba.jit(forward_model_ode)
forward_model_euler = numba.jit(forward_model_euler)

In [9]:
%timeit -n 100 f(forward_model_ode)

49.4 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit -n 100 f(forward_model_euler)

5.62 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
